In [9]:
import pandas as pd
import datasets

In [18]:
df = pd.read_csv("data/gen_data_20251118_for_dpo.csv", encoding = "utf-8").\
    loc[lambda _df : _df.tie == "N"]
df_text = pd.read_csv("data/data_sample_20251111_01.csv", encoding = "cp949")
system_prompt = df_text.system[0]

In [27]:
ds = datasets.Dataset.from_pandas(df)
columns_to_remove = [f for f in list(ds.features) if f != "subject_id"]

In [28]:
train_ds = ds.map(
    lambda sample: {
        "prompt": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": sample["text"]}
            ],
        "chosen": [{"role": "assistant", "content": sample["chosen"]}],
        "rejected": [{"role": "assistant", "content": sample["rejected"]}]
    }
)

Map: 100%|██████████| 1979/1979 [00:00<00:00, 8943.37 examples/s]


In [29]:
train_ds.map(remove_columns = columns_to_remove, batched = False)
train_ds = train_ds.train_test_split(test_size = 0.1, seed = 42)

train_ds["train"].to_json("data/dpo_train_dataset.json", orient = "records")
train_ds["test"].to_json("data/dpo_test_dataset.json", orient = "records")

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 34.66ba/s]


4651377

In [30]:
test_ds = train_ds["test"]

In [35]:
test_ds[0]["chosen"][0]["content"]

'A female patient presented with chief complaint of back pain; on physical examination, BP 98/63, HR 77, RR 18, and Temp 97.6; admission labs showed WBC 7.7, RBC 4.53, Hgb 13.4, Hct 39.3, Plt 321, MCV 87, MCH 29.6, MCHC 34.1, RDW 12.2, and Glucose 103; and the most diagnostically relevant finding was a large intramural hematoma in the descending aorta with a type B dissection and up to 5.6 cm dilatation of the proximal descending aorta, visualized on CTA of the chest and abdomen.'

In [41]:
lst = []

for idx in range(test_ds.num_rows):
    lst.append({"subject_id": test_ds["subject_id"][idx], "chosen": test_ds[idx]["chosen"][0]["content"], "rejected": test_ds[idx]["rejected"][0]["content"], "text": test_ds[idx]["prompt"][1]["content"]})

pd.DataFrame(lst).to_excel("data/dpo_test_label.xlsx", index = False)